In [1]:
import torch
import torch.nn as nn
from torchvision import models
import os
import matplotlib.pyplot as plt
import cv2
from torchvision.transforms import v2
import wandb
from torch.optim import Adam, AdamW
from torch.optim.lr_scheduler import MultiStepLR
from tqdm import tqdm
from pathlib import Path
from torch.utils.data import Dataset, DataLoader

In [2]:
lr = 0.001
EPOCH_NUM = 40
BATCH_SIZE = 64
weight_decay = 1e-4

In [3]:
class ImageTransform:
    def __init__(self, kind):
        if kind == 'train':
            self.transform = v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True),
                                      v2.RandomHorizontalFlip(), v2.RandomVerticalFlip(), v2.GaussianNoise(sigma=0.01),
                                      v2.RandomRotation(15), 
                                      v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])
            
        else:
            self.transform = v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True),
                                      v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])


    def __call__(self, sample):
        return self.transform(sample)


class CustomImageDataset(Dataset):
    def __init__(self, path, kind, transform=None, target_transform=None):
        self.prepare_image_path(path, kind)
        self.transform = transform
        self.target_transform = target_transform

    def prepare_image_path(self, path, kind):
        dataset_path = os.path.join(path, kind)
        if kind == "train":
            self.class_to_idx = {}
            image_path_list = []
            label_list = []
            class_list = os.listdir(dataset_path)
            class_list.sort()
            for i, cls in enumerate(class_list): 
                self.class_to_idx[cls] = i
                cls_images_path = os.path.join(dataset_path, cls + '/images')
                image_list = os.listdir(cls_images_path)
                for image_name in image_list:
                    image_path_list.append(os.path.join(cls_images_path, image_name))
                label_list += [i] * len(image_list)

        elif kind == "val":
            image_path_list = []
            label_list = []
            class_list = os.listdir(dataset_path)
            class_list.sort()
            for i, cls in enumerate(class_list):# to work properly, needed to create map name to int, using train info data
                cls_images_path = os.path.join(dataset_path, cls)
                image_list = os.listdir(cls_images_path)
                for image_name in image_list:
                    image_path_list.append(os.path.join(cls_images_path, image_name))
                label_list += [i] * len(image_list)

        elif kind == "test":
            image_path_list = []
            images_path = os.path.join(dataset_path, 'images')
            image_list = os.listdir(images_path)
            for image_name in image_list:
                image_path_list.append(os.path.join(images_path, image_name))
            label_list = None

        else:
            raise Exception("wrong kind argument!!!")
    
        self.image_path_list = image_path_list
        self.label_list = label_list

    def __len__(self):
        return len(self.image_path_list)

    def __getitem__(self, idx):
        image = cv2.imread(self.image_path_list[idx])
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        if self.label_list:
            label = self.label_list[idx]
        else:
            label = -1

        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label


def get_dataloader(path, kind):
    dataset = CustomImageDataset(path, kind, ImageTransform(kind), torch.tensor)
    shuffle = True if kind == 'train' else False
    return DataLoader(dataset, batch_size=BATCH_SIZE, num_workers=8, shuffle=shuffle)

In [4]:
from urllib.request import urlretrieve

def download_dataset(path, url='http://cs231n.stanford.edu/tiny-imagenet-200.zip'):
    dataset_name = 'tiny-imagenet-200'

    if os.path.exists(os.path.join(path, dataset_name, "val", "n01443537")):
        print("%s already exists, skipping download" % os.path.join(path, dataset_name))
        return
    elif not os.path.exists(os.path.join(path, 'tiny-imagenet-200' + ".zip")):
        print("Dataset doesn't exist or is broken, downloading it")
        urlretrieve(url, os.path.join(path, dataset_name + ".zip"))

    import zipfile
    with zipfile.ZipFile(os.path.join(path, 'tiny-imagenet-200' + ".zip"), 'r') as archive:
        archive.extractall()

    # move validation images to subfolders by class
    val_root = os.path.join(".", dataset_name, "val")
    with open(os.path.join(val_root, "val_annotations.txt"), 'r') as f:
        for image_filename, class_name, _, _, _, _ in map(str.split, f):
            class_path = os.path.join(val_root, class_name)
            os.makedirs(class_path, exist_ok=True)
            os.rename(
                os.path.join(val_root, "images", image_filename),
                os.path.join(class_path, image_filename))

    os.rmdir(os.path.join(val_root, "images"))
    os.remove(os.path.join(val_root, "val_annotations.txt"))

In [5]:
AUX_DATA_ROOT = Path(".")
download_dataset(AUX_DATA_ROOT)

./tiny-imagenet-200 already exists, skipping download


In [6]:
root_datasets = "./"

train_dataloader = get_dataloader(f"{root_datasets}/tiny-imagenet-200/", 'train')
val_dataloader   = get_dataloader(f"{root_datasets}/tiny-imagenet-200/", 'val')
test_dataloader = get_dataloader(f"{root_datasets}/tiny-imagenet-200/", 'test')

In [7]:
wandb.login(key="c86834f3fe7719b70c289274009689f98e6f5c1d")

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


wandb: Currently logged in as: bogdan_aleksandrov (bogdan_aleksandrov-no). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/b0gcham5/.netrc


True

In [8]:
wandb.init(
    # set the wandb project where this run will be logged
    project="aim_ml3_hw2",

    # track hyperparameters and run metadata
    config={
    "learning_rate": lr,
    "architecture": "resnet34",
    "dataset": "tiny-imagenet-200",
    "epochs": EPOCH_NUM,
    "batch_size": BATCH_SIZE,
    "optim": "Adam",
    "transform": "noise+scheduler"
    }
)

In [7]:
model = models.resnet50(weights=None, num_classes=200)
model.to('cuda')
pass

In [10]:
optimizer = Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
criterion = nn.CrossEntropyLoss()
scheduler = MultiStepLR(optimizer, milestones=[15, 25], gamma=0.5)

In [11]:
prev_val_acc = 0
for i in range(EPOCH_NUM):
    total_acc = 0
    total_loss = 0
    total_n = 0
    model.train()
    for X, y in tqdm(train_dataloader):
        X = X.to('cuda')
        y = y.to('cuda')

        out = model(X)
        loss = criterion(out, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        accuracy = (out.argmax(1) == y).sum().item()
        total_acc += accuracy
        total_loss += loss.item() * X.shape[0]
        total_n += X.shape[0]

    train_acc = total_acc / total_n
    train_loss = total_loss / total_n

    total_acc = 0
    total_loss = 0
    total_n = 0
    model.eval()
    with torch.no_grad():
        for X, y in val_dataloader:
            X = X.to('cuda')
            y = y.to('cuda')

            out = model(X)
            loss = criterion(out, y)
            accuracy = (out.argmax(1) == y).sum().item()

            total_acc += accuracy
            total_loss += loss.item() * X.shape[0]
            total_n += X.shape[0]

    if scheduler:
        scheduler.step()
    wandb.log({"train_acc": train_acc, "train_loss": train_loss, 
               "val_acc": total_acc / total_n, "val_loss": total_loss / total_n})

    if (total_acc / total_n > prev_val_acc):
        prev_val_acc = total_acc / total_n
        torch.save({
            'epoch': i,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'acc': prev_val_acc,
            }, 'checkpoint.pth')

  0%|          | 0/1563 [00:00<?, ?it/s]

 18%|█▊        | 277/1563 [02:45<12:49,  1.67it/s]


KeyboardInterrupt: 

In [12]:
wandb.finish()

train_acc,▁█
train_loss,█▁
val_acc,▁█
val_loss,█▁
train_acc,0.14002
train_loss,3.93015
val_acc,0.1646
val_loss,3.86756


In [7]:
import pandas as pd
from solution import predict

Defaulting to user installation because normal site-packages is not writeable


In [25]:
model = models.resnet50(weights=None, num_classes=200)
pass

In [27]:
w = torch.load('checkpoint_17.pth', weights_only=True)

In [29]:
map_classes = {class_idx: class_name for class_name, class_idx in train_dataloader.dataset.class_to_idx.items()}

In [30]:
pred_dict = {}
pred_labels = []
model.eval()
model.to('cuda')
for batch, _ in tqdm(test_dataloader):
    with torch.no_grad():
        _, predicted_labels = predict(model, batch).max(1)
    pred_labels.extend(predicted_labels.tolist())

for i, img_name in enumerate(test_dataloader.dataset.image_path_list):
    pred_dict[img_name.split("/")[-1]] = map_classes[pred_labels[i]]

100%|██████████| 157/157 [00:08<00:00, 18.64it/s]


In [31]:
submission_df = pd.DataFrame(pred_dict.items(), columns=["id", "pred"])
submission_df.to_csv("submission14.csv", index=False)

In [32]:
!md5sum checkpoint.pth

1089d094ad7bf904c14058d4ff7103b6  checkpoint.pth
